# About

In this advanced example we demonstrate how client-side encryption can work in Chroma. This is a useful technique if the user is willing to trade off restricted functionality for increased security.

In the below example user will not be able to search the text in the document using `$contains` filter with `where_document`.

In [1]:
# Install cryptography (required) and essential-generators (optional, for generating random sentences te demonstrate the functionality)
!pip install cryptography essential-generators

In [2]:
import uuid
from essential_generators import DocumentGenerator
from cryptography.fernet import Fernet
from chromadb import Settings
import chromadb

client = chromadb.Client(settings=Settings(allow_reset=True))
client.reset()
gen = DocumentGenerator()
key = Fernet.generate_key()
f = Fernet(key)

encrypt  = lambda x: f.encrypt(x.encode()).decode()
col = client.get_or_create_collection("test1")

entries = 10
encrypted_docs = [encrypt(gen.sentence()) for _ in range(entries)]
print("Encrypted documents: ",encrypted_docs)
col.add(ids=[f"{uuid.uuid4()}" for _id in range(entries)],embeddings=[[1,2,3] for _ in range(entries)],documents=encrypted_docs,metadatas=[{"test":"test"} for _ in range(entries)])

Encrypted documents:  ['gAAAAABk_GJCeohCgHQxjZeXt7AryHzgXTAZyXMrXJtu2H7hLz3d7OdL8qyEkIGMGhBmkUIetsY9D1uJOmCov3vCAfqHUfn_Zq4bCIR-dl8Qs-SspO7Fp3NC-v5c-Na2pCi3kvl_gODL9hVZVxlaX2GhcrLkOfbMTA==', 'gAAAAABk_GJDL34rIgei_mip0SPINA61SPsyiKQ1mnS1GGA-tOrU3XZa-db5Ab4Ly_64-Cpa4dKVuFptsToEXK9BYMRZMKhttkmBjhgzJNppjut28rQZhiZ4Ps5veWyzx3BMF2sC5VOVbKuhZN2esVLH17lDBrwfROEw-RWAyETyrUvYqyRhyHrC-jaU4mp0klptkNTL7m5g4XajSanfDkrZ55W_UsRyrQ==', 'gAAAAABk_GJDcpUOxomev-fmwNf2UZI_zC13M3IEMfHOBlnKUILcF61Vp4hPz06BPQiiQoEMssVrASVRHxBeTpTaG_pbzNz2qO58wsXOTknfzflv7S7kS-QK0_-6QixKK62fymIQIqM5', 'gAAAAABk_GJDKR58PZ-loDknZyRszFOGlLSY3KpsZ8cVLUtggw8LUzY4ThLvooPKXC7Qau7F_XTHctNV7x0643wj0ScdEIIA3USmflk9wrPifdkQrOQvGkCq6nCzLLrGZ4VitIfF5UFMKzVy9QQ42W_HDV9gaTvkGKO5SCMPVsUAXkL3AVdk6wA=', 'gAAAAABk_GJD7lqa3FpitWmW2GKJcDN06KH8EftEfP3tGS7C0AkSXd-VtoImPCmkn8mzlAQRemYrmPMJlezFAAconED4ow5-FdRRQiUQo-o4N5BOfwZOA8wFbQgo69rJSc_AILrBr-0QOzm8rwlJAi4ztFY3L4ZVsiSUW72b5d_S4WJk67SoEOc=', 'gAAAAABk_GJD1RHpr1F6Zx5-flBP8py1BRUNzsmx1ed0DMOEfCisxWTX

In [3]:
# Note if your data has been encrypted with different key or is in plain text the decryption will fail
decrypt = lambda x: f.decrypt(x.encode()).decode()
collection_with_encrypted_docs = client.get_or_create_collection("test1")
print("Decrypted documents: ",[decrypt(x) for x in collection_with_encrypted_docs.get()['documents']])

Decrypted documents:  ['Temperature range regular civil and criminal law are valid', 'Fixed to traffic control around a construction site next to each other. Social psychologists study', 'Californians are America". Every', 'Or quarry. and general goods imported from United States and China, the', 'Sun (at directions. Ethics implicitly regulates areas and grade levels tested.', 'Same ratio traditional values and cultural norms. They', 'And arrived influence because of', 'To cataplexy, planet. Models have been banned from playing Gaelic football.', 'A cognitive to 1972, and the', 'Schools (CPS) sampling in quality']


Entries in DB are also encrypted

Note: The below example was taken from client/server by examining the chroma.sqlite3 file.

![Encrypted DB](assets/client-side-encrypted-db-docs.png)

In [5]:
!python -c "from cryptography.fernet import Fernet; print(Fernet.generate_key().decode())" > symmentric_encryption.key

# Future Work

It is possible to also create a SSE (Searchable Symmetric Encryption) implementation, but it comes with the following drawbacks:

- Complex to implement correctly.
- Potential for information leakage, depending on the scheme (e.g., revealing access patterns or search patterns).
- Not as secure as traditional encryption in terms of hiding all data access patterns.
